                 # Hilda Indriani Gea - Data Engineering  
               13 September 2021 Take-Home Assement Zalora

1.
SELECT store_name, rnk from
(SELECT store_name, rev, RANK()OVER (ORDER BY rev desc, store_name) AS rnk FROM
(SELECT store_name, slot_member+slot_guest rev FROM
(SELECT a.name store_name, b.slots*a.membercost slot_member, b.slots*a.guestcost slot_guest FROM
(SELECT * FROM stores) a 
JOIN
(SELECT * FROM reservation where memid = 0 having year(starttime) = '2020') b
on a.facid = b.facid) a) a) a where rnk < 4

2. 
SELECT store_name, rev,
case
when rev between 0 and 10000000 then 'LOW'
when rev between 10000001 and 30000000 then 'AVERAGE'
when rev > 30000000 then 'HIGH'
end as classification  FROM
(SELECT store_name, slot_member+slot_guest rev FROM
(SELECT a.name store_name, b.slots*a.membercost slot_member, b.slots*a.guestcost slot_guest FROM
(SELECT * FROM stores) a 
JOIN
(SELECT * FROM reservation ) b
on a.facid = b.facid) a) a ORDER BY classification , store_name

3. 
SET @start_date = DATE_ADD(CURRENT_DATE, INTERVAL -15 DAY);
SET @end_date = DATE_ADD(CURRENT_DATE, INTERVAL -1 DAY);

SELECT date,
ROUND(SUM(CASE WHEN date between @start_date and @end_date then rev/15 end),2) 'rev'
FROM
(SELECT date, slot_member+slot_guest rev FROM
(SELECT date(starttime) date,  b.slots*a.membercost slot_member, b.slots*a.guestcost slot_guest FROM
(SELECT * FROM stores) a 
JOIN
(SELECT * FROM reservation having date(starttime) between '2020-08-01'  and '2020-08-31') b
on a.facid = b.facid) a) a GROUP BY 1 order by date

4.
with recursive 
mem as (
select  d.* from member d where memid = 15
union all
select d.* from member d inner join mem c on c.memid = d.recomended_by
),
rec as (
select  d.* from member d where memid = 15
union all
select d.* from member d inner join rec p on d.memid = p.recomended_by
)
select * from mem
union select * from rec
order by memid
